In [ ]:
!pip install 'farm-haystack[all]'
!pip install pytorch-lightning==1.7.7
!pip install spacy
!python -m spacy download en_core_web_sm
!pip install transformers

In [1]:
from utils import get_all_files_with_extension

In [ ]:
from haystack.utils import launch_milvus()
launch_milvus()

In [2]:
from haystack.document_stores import MilvusDocumentStore
#document_store = FAISSDocumentStore(sql_url="sqlite:///data/my_doc_store.db",faiss_index_factory_str="Flat",similarity="dot_product", embedding_dim=128)

document_store = MilvusDocumentStore(sql_url="sqlite:///data/my_doc_store.db",similarity="dot_product", embedding_dim=128, progress_bar= True)

json_locations = "data/extract_json"

all_json_files = get_all_files_with_extension(json_locations, "json")

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/espnet2/gan_tts/vits/vits.py:43: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(torch.__version__) >= LooseVersion("1.6.0"):


In [3]:
import json

documents = []
for i in all_json_files:
    with open(i,"r") as f:
        document = json.load(f)
    
    for j in document["paras"]:
        dict_ = {
            'content' : j,
            'meta': {
                'file_name' : document['file_name'],
                'abstract' : document['abstract'],
                'authors' : document['authors'] if 'authors' in document else None,
                'title': document['title']  if 'title' in document else None
                
            }
        }
        documents.append(dict_)
    

In [ ]:
document_store.write_documents(documents)

  0%|          | 0/996193 [00:00<?, ?it/s]

In [3]:
import torch
checkpoint = torch.load("trained_model/epoch=0-step=49049.ckpt",map_location=torch.device('cpu'))

In [4]:
from model import Retriever
from train_sagemaker import TrainConfig

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

config = TrainConfig()
model = Retriever(config).to(device)


if device.type == 'cpu':
    model.load_state_dict(checkpoint["state_dict"],map_location=torch.device('cpu'))
else:
    model.load_state_dict(checkpoint["state_dict"])
    
    
model.eval()

/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):


cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.se

Retriever(
  (query_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

In [5]:
from haystack.nodes import DensePassageRetriever
from transformers import AutoTokenizer
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from functools import partial
from tqdm.auto import tqdm

import numpy as np


'''
Building a custom retriever because the implementation in haystack wants the passage and query encode to be disjoint 
models with their tokenizers beings accessile with the huggingface path or local path

Referred from : https://github.com/deepset-ai/haystack/blob/main/haystack/nodes/retriever/dense.py

Missing functionalities:
    1. using filters for metadata
    2. batch retrieve
'''


class CustomDataset(Dataset):
    def __init__(self, text):
        self.data = text
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    

def collate_fn(passages, tokenizer, passage_max_len):
    tokenized_passages = tokenizer(passages, padding= True, truncation= True, max_length=passage_max_len, return_tensors="pt")
    
    return tokenized_passages
    

class CustomDensePassageRetriever():
    def __init__(
        self,
        document_store,
        model,
        query_tokenizer_loc,
        passage_tokenizer_loc,
        max_seq_len_query: int = 64,
        max_seq_len_passage: int = 288,
        top_k: int = 10,
        use_gpu: bool = True,
        batch_size: int = 16,
        scale_score: bool = True,
    ):
        self.document_store = document_store
        self.model = model
        self.query_tokenizer = AutoTokenizer.from_pretrained(query_tokenizer_loc)
        self.passage_tokenizer = AutoTokenizer.from_pretrained(passage_tokenizer_loc)
        
        self.max_seq_len_query = max_seq_len_query
        self.max_seq_len_passage = max_seq_len_passage
        self.topk = top_k
        self.batch_size = batch_size
        
        self.scale_score = scale_score
        
    def embed_documents(self, documents):
       
        """
        # possible speed up by using Dataloaders??
        text = [i["content"] for i in documents]
        
        dataset = CustomDataset(text)
        collate_fn = partial(collate_fn, self.passage_tokenizer, self.max_seq_len_passage)
        
        datalaoder = DataLoader(dataset, batch_size= self.batch_size, collate_fn=collate_fn_)
        """
        
        document_embeddings = []
        
        with tqdm(range(len(documents)), desc= "Creating document embeddings", unit="Docs") as progress_bar:
            for i in range(0, len(documents), self.batch_size):
                text_ = documents[i:i+self.batch_size]
                
                text = [i.content for i in text_]

                tokenized_passages = self.passage_tokenizer(text, padding= True, truncation= True, \
                                                            max_length=self.max_seq_len_passage, return_tensors="pt").to(device)

                with torch.no_grad():
                    encoded_passage = self.model.passage_encoder(**tokenized_passages)
                    encoded_passage_representation = encoded_passage.pooler_output

                    if self.model.config.projection:
                        encoded_passage_representation = self.model.fc_passage(encoded_passage_representation)

                    encoded_passage_representation = self.model.layer_norm_passage(encoded_passage_representation)

                document_embeddings.append(encoded_passage_representation.cpu().numpy())
                progress_bar.update(self.batch_size)
        
        return np.concatenate(document_embeddings, axis=0)
        
        

    def embed_queries(self, queries):
        
        
        query_embeddings = []
        for i in range(0, len(queries), self.batch_size):
            text = queries[i:i+self.batch_size]
            
            tokenized_queries = self.query_tokenizer(text, padding= True, truncation= True,
                                                     max_length=self.max_seq_len_query, return_tensors="pt").to(device)

            with torch.no_grad():
                encoded_query = self.model.query_encoder(**tokenized_queries)
                encoded_query_representation = encoded_query.pooler_output

                if self.model.config.projection:
                    encoded_query_representation = self.model.fc_query(encoded_query_representation)

                encoded_query_representation = self.model.layer_norm_query(encoded_query_representation)

                query_embeddings.append(encoded_query_representation.cpu().numpy())
                
        return np.concatenate(query_embeddings, axis=0)
        

    def retrieve(self,query,
                    top_k = None,
                    index = None,
                    headers = None,
                    scale_score = None,
                    document_store = None,
                     filters = None
                    ):

            document_store = document_store or self.document_store

            assert document_store != None

            if top_k is None:
                top_k = self.topk

            if index is None:
                index = document_store.index

            if scale_score is None:
                scale_score = self.scale_score

            query_emb = self.embed_queries(queries=[query])

            documents = document_store.query_by_embedding(
                    query_emb=query_emb[0], top_k=top_k, filters=filters, index=index, headers=headers, scale_score=scale_score
                )

            return documents

In [6]:
retriever = CustomDensePassageRetriever(document_store=document_store,
                                        model=model,
                                        query_tokenizer_loc='bert-base-uncased',
                                        passage_tokenizer_loc='bert-base-uncased',
                                        top_k = 5,
                                        batch_size = 768)

In [ ]:
document_store.update_embeddings(retriever)

Updating Embedding:   0%|          | 0/994444 [00:00<?, ? docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

...

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

Creating document embeddings:   0%|          | 0/10000 [00:00<?, ?Docs/s]

In [ ]:
document_store.save(index_path="/home/ec2-user/SageMaker/arxiv_hunter/data/saved_store")

In [12]:
for doc in retriever.retrieve("Attention NLP decoder"):
    print(doc.meta["title"])
    print(doc.meta['file_name'])
    print(doc.content)

Sketch-Based Anomaly Detection in Streaming Graphs
2106.04486v2.pdf.txt
This ensures that the current submatrix is as condensed as possible (line 9). As defined in Definition 2, AnoEdge-L computes the likelihood score of the edge with respect to (𝑆𝑐𝑢𝑟,𝑇𝑐𝑢𝑟) (line 10). A higher likelihood measure implies that the edge is more likely to be anomalous. Algorithm 3: AnoEdge-L : Streaming Anomaly Edge Scor- ing Input: Stream ℰ of edges over time Output: Anomaly score per edge 1 Procedure AN OEDGE-L(ℰ) /* H-CMS data structure */ 2 Initialize H-CMS matrix M for edges count /* mutable submatrix */ 3 Initialize a randomly picked 1 × 1 submatrix (𝑆𝑐𝑢𝑟,𝑇𝑐𝑢𝑟) 4 while new edge 𝑒 = (𝑢, 𝑣,𝑤,𝑡) ∈ ℰ is received do /* decay count */ 5 Temporal decay H-CMS with timestamp change 6 Update H-CMS matrix M for new edge (𝑢, 𝑣) with value 𝑤 // update count 7 ⊲ Check and Update Submatrix: 8 Expand (𝑆𝑐𝑢𝑟,𝑇𝑐𝑢𝑟) // expand submatrix 9 Condense (𝑆𝑐𝑢𝑟,𝑇𝑐𝑢𝑟) // condense submatrix /* likelihood score from Definition 2 */